## Graduation Project

 The objective is to create an agent (or a function), that explores a grid, to cover every available square with a valid path, using for example `dfs` or `bfs` while avoiding the walls which function as obstacles.

If this is something you're interested in, definitely try to code a functions that does this, or even one that is able to do it efficiently and optimally! This is a much more open project than our typical labs, but one possible solution is provided. Many new concepts are involved in this project, including `nonlocal` scope, `variable` number of arguments, using `sets`, nested functions, etc. but I hope you guys will be able to learn and explore it on your own without any TAs (with the help of best friend Google)

If you have any problems with it (or even any solutions!), feel free to text me or any of the other TAs. There is a lot less guidance provided in this notebook, but there is a potential solution included below under `Staff Solution`.

If you do decide to take up the challenge, I wish you good luck and all the best!

In [11]:
!pip install colorama --quiet
from IPython.display import clear_output, HTML, display
import random
from colorama import Fore, Back, Style
from time import sleep, time
from functools import partial
from threading import Timer # potential add on if we want to fix the input bug for regular play
print = partial(print, flush=True)

## Important Helper Functions and Constants (do not modify)

In [20]:
ANSI_COLOR_CHARACTERS = {
    "black": Fore.BLACK,
    "blue": Fore.BLUE,
    "cyan": Fore.CYAN,
    "green": Fore.GREEN,
    "grey": Fore.LIGHTBLACK_EX,
    "light_blue": Fore.LIGHTBLUE_EX,
    "light_cyan": Fore.LIGHTCYAN_EX,
    "light_green": Fore.LIGHTGREEN_EX,
    "light_magenta": Fore.LIGHTMAGENTA_EX,
    "light_red": Fore.LIGHTRED_EX,
    "light_white": Fore.LIGHTWHITE_EX,
    "light_yellow": Fore.LIGHTYELLOW_EX,
    "magenta": Fore.MAGENTA,
    "red": Fore.RED,
    "white": Fore.WHITE,
    "yellow": Fore.YELLOW,
}

def colored(string, color):
  assert color in ANSI_COLOR_CHARACTERS, f"We don't have a color named '{color}'. Please use one of the following colot names: {list(ANSI_COLOR_CHARACTERS.keys())}"
  ansi_char = ANSI_COLOR_CHARACTERS[color]
  return ansi_char + string + Fore.RESET

WIN = "WIN"
CONTINUE = "CONTINUE"
LOSE = "LOSE"
MOVES = [(0,1),(1,0),(-1,0),(0,-1)]
EMPTY_CELL = "*"
EMPTY_CELL = "⬜"
WALLS = "⬛"

CURRENT = "🏃‍♂️"
PATH_HISTORY = "👣"
STANDARD_ERROR = "Your move is not valid!"

def center_output():
  display(HTML("""
  <style>
  #output-body {
      display: flex;
      align-items: center;
      justify-content: center;
  }
  </style>
  """))


import errno
import os
import signal
import functools

class TimeoutError(Exception):
    pass

def timeout(seconds=10, error_message=os.strerror(errno.ETIME)):
    """
    Use the @timeout decorator to stop a function after it exceeds some number of seconds
    """

    def decorator(func):
        def _handle_timeout(signum, frame):
            raise TimeoutError(error_message)

        @functools.wraps(func)
        def wrapper(*args, **kwargs):
            signal.signal(signal.SIGALRM, _handle_timeout)
            signal.alarm(seconds)
            try:
                result = func(*args, **kwargs)
            finally:
                signal.alarm(0)
            return result

        return wrapper

    return decorator

In [21]:

def get_grid_string(grid):
  """
    Converts grid (2d list) to a single string
  """
  return ''.join(item for row in grid for item in row)

def get_move(char):
  """
    Returns the movement in the grid based on input
  """
  if char == 'w': return (-1,0)
  if char == 'a': return (0,-1)
  if char == 's': return (1,0)
  if char == 'd': return (0,1)
  return 0,0

def check_state(grid, curr_row, curr_col):
  """
  Takes in the grid and the most recent move
  Returns a game state of (WIN, CONTINUE, LOSE)
  """
  
  # win if there are no more '.' in the grid
  grid_string = get_grid_string(grid)
  if EMPTY_CELL not in grid_string: return WIN

  for x,y in MOVES:
    r,c = curr_row + x, curr_col + y
    if 0 <= r < len(grid) and 0 <= c < len(grid[0]) and grid[r][c] == EMPTY_CELL:
      return CONTINUE
  
  # lose if no more moves available
  return LOSE


def print_grid(grid):
  """
    Prints grid in a human readable fashion
  """
  string = ''
  for i in range(len(grid)):
    string += ' '.join(grid[i]) + '\n'
  print(string, flush=True)

def create_grid(rows, columns):
  """ 
    Create a grid of rows x columns, populated with walls
  """
  return [[WALLS] * columns for _ in range(rows)]


def generate_random_path(grid,path_length):
  """ 
    Generates a random path of path_length in the grid
  """

  rows, cols = len(grid), len(grid[0])
  if path_length >= rows*cols:
    print("Invalid input, path length cannot exceed cells in grid!", flush=True)
    return
  result_path = []
  visited = set()
  def dfs(row, col, path):
    if not ((0 <= row < rows) and (0 <=  col < cols)): return False
    if (row,col) in visited: return False
    path.append((row,col))
    visited.add((row,col))
    if len(path) == path_length: return True
    random.shuffle(MOVES)

    for x,y in MOVES:
      if dfs(row+x,col+y, path):
        return True
    path.pop()
    visited.remove((row,col))
    return False

  dfs(0,0,result_path)
  return result_path

def make_grid_path(grid, path):
  """ 
    Make all the cells along the path in the grid empty
  """
  modified_grid = [i[:] for i in grid]
  for x,y in path:
    modified_grid[x][y] = EMPTY_CELL
  return modified_grid



def create_level(grid_rows, grid_cols, path_length=None,seed=None,verbose=True):
  """
    Creates a random grid level
    Args:
        grid_rows (int) : number of rows in the grid
        grid_cols (int) : number of cols in the grid
        (optional) path_length (int): the number of empty cells
        (optional) seed (int): to seed the RNG to generate a specific grid
        (optional) verbose (bool): to print out the grid level characteristics
    Returns:
        grid_level (List(List[str])): the grid level with the empty cells
        seed (int): random seed used to generate this level
  """

  if not seed: seed = random.randint(0,10000000)
  random.seed(seed)
  MOVES.sort()

  assert grid_rows > 0 and grid_cols > 0, "Please give valid input... 😒😒😒"
  grid = create_grid(grid_rows, grid_cols)

  if not path_length: path_length = random.randint(5,grid_rows*grid_cols - 5)
  path = generate_random_path(grid, path_length)

  if verbose: print(f"Random seed: {seed}\n")
  grid_level = make_grid_path(grid, path)

  if verbose: print_grid(grid_level)

  return grid_level, path,seed

def check_valid(grid, grid_path):
  """ 
    Checks if the solution path is valid
    Args:
        grid (List(List[str])) : grid level
        grid_path (List(Tuple[int])): possible solution path
    Returns:
        a boolean if the given grid_path is a valid solution
  """

  rows,cols = len(grid), len(grid[0])
  row,col = 0,0
  if row != grid_path[0][0] or col != grid_path[0][1]: return False

  v = set()
  v.add((0,0))
  for i in range(1, len(grid_path)):
    rr,cc = grid_path[i]
    if (rr,cc) in v: return False

    if not ((0 <= rr < rows) and (0<= cc < cols)): return False
    if abs(row - rr) + abs(col - cc) != 1: return False
    if grid[rr][cc] != EMPTY_CELL:
      return False
    v.add((rr,cc))
    row,col = rr,cc

  grid_string = get_grid_string(grid)
  return grid_string.count(EMPTY_CELL) == len(v)

def exploration_animation(grid, grid_path):
  """ 
    Prints an animation of the exploration of the grid using the grid_path if it is valid
    Args:
        grid (List(List[str])) : grid level
        grid_path (List(Tuple[int])): solution path
    Returns:
        None
  """

  if not check_valid(grid, grid_path):
    print("Invalid solution!")
    return
  grid_copy = [i[:] for i in grid]
  for idx, (row,col) in enumerate(grid_path):
    sleep(1)
    if idx != 0:
      old_row, old_col = grid_path[idx-1]
      grid_copy[old_row][old_col] = PATH_HISTORY
    clear_output(wait=True)
    grid_copy[row][col] = CURRENT
    print_grid(grid_copy)
  clear_output(wait=True)
  grid_copy[row][col] = PATH_HISTORY
  print_grid(grid_copy)

  print("CONGRATULATIONS!")

In [22]:
def play(grid):
  """ 
    Plays the game of the grid level using the python input
  """
  grid_copy = [i[:] for i in grid]
  grid_copy[0][0] = CURRENT
  row, col = 0,0
  rows, cols = len(grid), len(grid[0])
  error_message = ''

  while True:
    clear_output(wait=False)
    print_grid(grid_copy)

    state = check_state(grid_copy, row,col)
    if state == WIN:
      print("YOU WIN!")
      break

    if state == LOSE:
      print("YOU LOSE!")
      print("Play again?")
      break

    if error_message: print(error_message)

    move = input("Enter your move (WASD): ")
    move = move.lower()

    if move not in 'wasd' or len(move) != 1:
      error_message = f"Your move '{move}' is not a valid move."
      continue
    x,y = get_move(move)
    xx, yy = row+x, col+y
    if not ((0 <= xx < rows) and (0 <= yy < cols)):
      error_message = STANDARD_ERROR
      continue
    if grid_copy[xx][yy] != EMPTY_CELL:
      error_message = STANDARD_ERROR
      continue
    grid_copy[row][col] = PATH_HISTORY
    grid_copy[xx][yy] = CURRENT
    row,col = xx,yy
    error_message = ''

### Let's play the game!
With the code we have so far, we can play a simple game using inputs (WASD). The objective is to fill in all the blank squares on the grid. You may not retreat on your path or go through the walls.

A few things about the icons you will see
- `⬜`: empty square, you may step here
- `⬛`: a wall, you may not step here
- `🏃‍♂️`: current location
- `👣`: a place that you have stepped before

First we create our level which is generated randomly. 

```py
# 4x4 grid with 10 empty cells
grid_level, grid_path, seed = create_level(4,4,10)
```

A random generator is used to generate the grid. If you like a particular level, you can take note of the `seed` and pass it into the function that generates the grid `create_level`

In [5]:
grid_level, grid_path, seed = create_level(4,4, 10, 9809112)

Random seed: 9809112

⬜ ⬛ ⬛ ⬛
⬜ ⬜ ⬜ ⬛
⬛ ⬜ ⬜ ⬜
⬛ ⬜ ⬜ ⬜



### Then we play it!

> Note: there may be some bugs where the input box suddenly disappears. If that happens, just restart the cell and try again! (Or if you are brave enough to explore the inner workings of python, you can try to debug and fix it)

In [6]:
play(grid_level)

👣 ⬛ ⬛ ⬛
👣 👣 👣 ⬛
⬛ 🏃‍♂️ 👣 👣
⬛ 👣 👣 👣

YOU WIN!


Hopefully you managed to complete the level! Now that we know how to play it manually, let's try to code something that can solve it! We're familiar with Depth First Search (DFS), so your first challenge is to try and use that to find a valid path! 

A few pointers to take note
- We want to try to create a solution on the grid, but we don't want to modify the grid. Because of that, if you would like to modify or access the grid, please use `grid_copy` instead of `grid`
- We want to return our solution path, so for every step you take, please add it to the `path` list.

E.g. If I start from `(0,0)` and move to the right once and move down once, my `path` variable should contain

```py
path = [(0,0), (0,1), (1,1)]
```

> P.S. `(0,0)` is known as a tuple, just take them as lists that cannot be modified! And to find out more, remember that Google is your best friend

## Q1: Code an agent to solve the game

### Your solution

In [7]:
def student_agent(grid):
  # given an input of a grid
  # write an algorithm to return a path that can cover all empty cells starting from the top left cell
  
  grid_copy = [i[:] for i in grid]
  path = []
  
  # YOUR CODE HERE


  return path

### Staff Solution

If you would like to view a sample solution, you can take a look here. However, this solution is very much not optimized, and can only pass a few of the test cases that we have. Try to do better than what we have shown you here!

In [8]:
# basic dfs answer with no optimization
def staff_agent(grid):
  # given an input of a grid
  # write an algorithm to return the correct path
  grid_copy = [i[:] for i in grid]
  path = []

  # YOUR CODE STARTS HERE
  row,col = 0,0
  path.append((row,col))
  rows,cols = len(grid), len(grid[0])
  count = get_grid_string(grid).count(EMPTY_CELL)
  grid_copy[0][0] = PATH_HISTORY

  def dfs(curr_row, curr_col):
    nonlocal count
    count -= 1
    if count == 0:
      return True

    for x,y in MOVES:
      rr, cc = curr_row + x, curr_col + y
      if not (0 <= rr < rows and 0 <= cc < cols): continue
      if grid_copy[rr][cc] != EMPTY_CELL: continue
      grid_copy[rr][cc] = PATH_HISTORY
      path.append((rr,cc))
      if dfs(rr,cc): return True
      path.pop()
      grid_copy[rr][cc] = EMPTY_CELL

    count += 1
    return False

  dfs(0,0)
  # YOUR CODE ENDS HERE

  return path


## Check your answer

In [9]:
def test_agent(dfs_function):
  # grid_level, grid_path, seed = create_level(4,4, 10, 9809112) -> I like testing on this grid

  # feel free to change the level that you would like to test on
  # this tests on a 4 by 4 grid with 10 empty cells (so 6 walls)
  grid_level, grid_path, seed = create_level(4,4, 10)
  path = dfs_function(grid_level)
  exploration_animation(grid_level, path)


In [10]:
for _ in range(2):
  # test_agent(student_agent) # UNCOMMENT THIS LINE ONCE YOU"RE DONE
  test_agent(staff_agent) # COMMENT THIS LINE OUT ONCE YOU"RE DONE

👣 ⬛ 👣 👣
👣 👣 👣 👣
⬛ ⬛ 👣 👣
⬛ ⬛ 👣 ⬛

CONGRATULATIONS!


Well done! You managed to complete this challenge, it definitely is not easy. However, here's a follow up challenge. How can you make your code even better, such that even if it is applied on big graphs, it will still find a solution in a reasonable time?

We have a few test cases below for you to try it out. See if you can find a solution for all the levels in just 5 seconds!

### Benchmarking

In [23]:
def benchmark(fn):
  levels = [
      (4,4, 10, 5599278,False),
      (4,4, 10, 9809112,False),
      (8,8, 38, 5599278, False),
      (8,8, 40, 8333500, False),
      (9,9, 40, 1555078, False),
      (9,9, 45, 1555078, False),
      (9,9, 50, 1555078, False),
      (9,9, 55, 1555078, False),
      (9,9, 60, 1555078, False),
      (10,10,60,1555078,False),
      (9,9, 60, 9773395, False),
      (9,9, 60,6418357,False),
      (10,10,70,7310160,False),
      (12,12, 80, 7599824, False),
      (12,12, 90, 8838895, False),
      (16,16, 80, 5443243, False),
      (16,16, 110, 4023681, False),
      
  ]
  t0 = time()
  solved_details = []
  time_given = 5
  
  @timeout(time_given)
  def test_fn():
    for idx, level in enumerate(levels):
      grid_level, grid_path, seed = create_level(*level)
      t1 = time()
      path = fn(grid_level)

      # verify solution
      if not check_valid(grid_level, path):
        print(f"Invalid path for level {idx+1}")
      else:
        t2 = time()
        solved_details.append(f"Solved level {idx+1} in {round(t2 - t1, 5)} seconds")
  
  try:
    test_fn()
  except Exception as e:
    # print(e)
    print(f"{time_given} seconds has elapsed")
  finally:
    t_last = time()
    print("Total time taken:", round(t_last - t0, 5), "seconds")
    print(f"Levels solved: {len(solved_details)} / {len(levels)}")
    print("Details:")
    for i in solved_details:
      print(i)

In [24]:
benchmark(staff_agent) # how many levels can this naive solution solve?
# benchmark(student_agent) # let's see if you can do better!

5 seconds has elapsed
Total time taken: 5 seconds
Levels solved: 10 / 17
Details:
Solved level 1 in 5.6743621826171875e-05 seconds
Solved level 2 in 2.9325485229492188e-05 seconds
Solved level 3 in 0.0009400844573974609 seconds
Solved level 4 in 0.0031652450561523438 seconds
Solved level 5 in 0.002752065658569336 seconds
Solved level 6 in 0.016485929489135742 seconds
Solved level 7 in 0.12607216835021973 seconds
Solved level 8 in 0.2293534278869629 seconds
Solved level 9 in 0.18251299858093262 seconds
Solved level 10 in 1.5828232765197754 seconds


### Note from Jon

By the time anyone sees this, it would be the end of JamCoders 2023 :'( ... its been such a blessed month here and I hope that this has been a meaningful month for every one of you.

I hope that for many of you, that it would not be the end of your coding journey. This was a potential project to be included at the end of week 3 but was neglected in favor of more exam prep. It relates to the game that Timnit was showing at the end of Week 2.

The objective is to create a function, that explores a grid, to cover every available square with a valid path, using for example `dfs` or `bfs` while avoiding the walls which function as obstacles.

If this is something you're interested in, definitely try to code a functions that does this, or even one that is able to do it efficiently and optimally! This is a much more open project than our typical labs, but one possible solution is provided. Many new concepts are involved in this project, including `nonlocal` scope, `variable` number of arguments, using `sets` etc. but I hope you guys will be able to learn it on your own without any TAs (with the help of best friend Google)

If you have any problems with it (or even any solutions!), feel free to text me or any of the other TAs. There is a lot less guidance provided in this notebook, but there is a potential solution included below under `Staff Solution`.

If you do decide to take up the challenge, I wish you good luck and all the best for your future!

Oh one extra thing, I wrote this question for the exam but it did not get used in the end, here it is, and see if you can solve it :))

```py
def unj(pawat, patwa):
    print(patwa)
    return pawat

def jit(wat,tana):
    print(wat + tana)

def pawat():
    return 'unjitwattana'

pawat() # Prints: __________________
print(jit(pawat(), pawat())) # Prints: __________________
print(unj(jit('pawat', 'patwa'), jit('patwa', 'pawat'))) # Prints: __________________
```